In [4]:
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
X = load_iris().data
y = load_iris().target
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [6]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# y_train = np.argmax(y_train, axis=1)
X_train= X_train.reshape(X_train.shape[0], -1)
X_test= X_test.reshape(X_test.shape[0], -1)

X_train = X_train /255
X_test = X_test / 255


a = np.zeros((y_train.shape[0], np.max(y_train)+1))
a[np.arange(len(a)), y_train] = 1
y_train = a

In [7]:

class ActivationFunction:
    def __init__(self):
        self.dictionary = {}
        self._create_dict()
        
    @staticmethod
    def sigmoid(z):
        z = np.copy(z)
        return 1.0/(1.0+np.exp(-z))
        
    def sigmoid_prime(self,z):
        z = np.copy(z)
        
        return self.sigmoid(z)*(1 - self.sigmoid(z))
    
    @staticmethod
    def relu(z):
        return np.maximum(0, z)
    
    @staticmethod
    def relu_prime(z):
        new_z = np.copy(z)
        new_z[ new_z > 0] = 1
        new_z[ new_z< 0] = 0
        return new_z
    
    @staticmethod
    def tanh(z):
        z = np.copy(z)
        
        return ((np.exp(z) - np.exp(-z))/
                (np.exp(z) + np.exp(-z)))
    
    def tanh_prime(self, z):
        z = np.copy(z)
        
        return 1- self.tanh(z)**2
    @staticmethod
    def soft_max(z):
        
        return np.exp(z)/ np.sum(np.exp(z))
    @staticmethod
    def soft_max_prime(z):
        
        return np.exp(z)/ np.sum(np.exp(z))

            
    
    
    def _create_dict(self):
        name = ['relu', 'sigmoid', 'tanh']
        child = [
            (self.relu, self.relu_prime),
            (self.sigmoid, self.sigmoid_prime),
            (self.tanh, self.tanh_prime),
        ]
        
        self.dictionary = dict(zip(name, child))
        

In [8]:
class Layer:
    def __init__(self, input_shape, output_shape,lr, n, activation="relu"):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.lr = lr
        self.n = n
        self.input = None
        self.output = None
        self.activation_type = activation
        self.activation = None
        self.activation_prime = None
        self.activation_func = ActivationFunction().dictionary
        self._choice_activation_func()
        
        self.weights = np.random.rand(self.input_shape, self.output_shape)
        self.biases = np.random.rand(1, self.output_shape)
        
    def forward_propagation(self, input):
        self.input = self.activation(input)
        
        if self.input.ndim ==1:
            self.input = self.input.reshape(1, -1)

        self.output = self.input@self.weights + self.biases
    

        return self.output
    def backward_propagation(self, prev_err):
        layer_err =  prev_err*self.activation(self.output)@self.weights.T
        
        d_weights = self.input.T@prev_err
        
        self.weights -= (self.lr*d_weights)
        self.biases -= (self.lr*np.sum(prev_err, axis=0))
        
        return layer_err
        
    def _choice_activation_func(self):
        activation_func = self.activation_func[self.activation_type]
        self.activation = activation_func[0]
        self.activation_prime = activation_func[1]

class LastLayer(Layer):
        
    def forward_propagation(self, input):
        self.input = np.array(input)
        self.output = self.input
        
        return np.exp(self.output) / np.sum(np.exp(self.output))
    
    def backward_propagation(self, prev_err):
            
        
        return self.activation_prime(self.input)*prev_err
    

In [9]:
class NetWork:
    def __init__(self, activation, layer: list[int], lr, epochs):
        self.activation = activation
        self.layers: list[Layer] = []
        self.num_layers = layer
        self.lr = lr
        self.epochs = epochs
        self.n = None
        
        
    def _create_layers(self):
        for i in range(len(self.num_layers)-1):
            layer = Layer(input_shape=self.num_layers[i],
                          output_shape=self.num_layers[i+1],
                          lr=self.lr,
                          activation=self.activation,
                          n=self.n
                          )
            self.layers.append(layer)
        last_layer = LastLayer(
            input_shape= self.num_layers[-1],
            output_shape= 1,
            lr= self.lr,
            activation=self.activation,
                          n=self.n
            
            
        )
        self.layers.append(last_layer)

    def fit(self, X,y):
        self.n = len(X)
        self._create_layers()
        for i in range(self.epochs):
            error_epoch = 0
            
            output = np.copy(X)
            for layer in self.layers:
                output = layer.forward_propagation(output)
            
            error_epoch += self.loss(y, output)
            
            error_prime = self.loss_prime(y, output)
            for layer in reversed(self.layers):
                error_prime = layer.backward_propagation(error_prime)
            
            error_epoch = np.sum(error_epoch) / self.n
            print(f'At epoch : {i} has error : {error_epoch}')
            
                   
    @staticmethod
    def loss(y_true, y_pred):
        return -np.sum(y_true * np.log(y_pred+ 1e-8))
    @staticmethod
    def loss_prime(y_true, y_pred):
        return y_true-y_pred
    
    def predict(self, X):
        res = []
        for x in X:
            output = x
            
            for layer in self.layers:
                output = layer.forward_propagation(output)
            res.extend(output)
        res = np.argmax(res, axis=1)
        return res

In [11]:
net = NetWork(activation='sigmoid', layer=[784, 16,16,  10], epochs=1000, lr=0.03)
net.fit(X_train[:100],y_train[:100])

At epoch : 0 has error : 7.540114230875877
At epoch : 1 has error : 7.541321017447178
At epoch : 2 has error : 7.54253289685025
At epoch : 3 has error : 7.543749908840249
At epoch : 4 has error : 7.544972093647385
At epoch : 5 has error : 7.546199491984623
At epoch : 6 has error : 7.547432145055583
At epoch : 7 has error : 7.5486700945625715
At epoch : 8 has error : 7.549913382714808
At epoch : 9 has error : 7.551162052236798
At epoch : 10 has error : 7.552416146376904
At epoch : 11 has error : 7.553675708916078
At epoch : 12 has error : 7.554940784176796
At epoch : 13 has error : 7.556211417032166
At epoch : 14 has error : 7.557487652915248
At epoch : 15 has error : 7.558769537828554
At epoch : 16 has error : 7.560057118353773
At epoch : 17 has error : 7.561350441661691
At epoch : 18 has error : 7.562649555522333
At epoch : 19 has error : 7.563954508315336
At epoch : 20 has error : 7.565265349040524
At epoch : 21 has error : 7.566582127328741
At epoch : 22 has error : 7.56790489345291

C:\Users\DELL\AppData\Local\Temp\ipykernel_19544\817588615.py:9: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


At epoch : 235 has error : 8.885075729889618
At epoch : 236 has error : 8.906323060264508
At epoch : 237 has error : 8.925379148780003
At epoch : 238 has error : 8.940131103247303
At epoch : 239 has error : 8.949384582790493
At epoch : 240 has error : 8.952833606712316
At epoch : 241 has error : 8.950662812143598
At epoch : 242 has error : 8.943223890148321
At epoch : 243 has error : 8.930881347972974
At epoch : 244 has error : 8.91397156065018
At epoch : 245 has error : 8.892820197128902
At epoch : 246 has error : 8.867791631218275
At epoch : 247 has error : 8.839362180561192
At epoch : 248 has error : 8.808214796319213
At epoch : 249 has error : 8.775346264413205
At epoch : 250 has error : 8.742154973659899
At epoch : 251 has error : 8.71043603237589
At epoch : 252 has error : 8.682174207411299
At epoch : 253 has error : 8.659071553876563
At epoch : 254 has error : 8.641959052941024
At epoch : 255 has error : 8.630491113028079
At epoch : 256 has error : 8.623414566731508
At epoch : 2

In [12]:
prediction = net.predict(X_test)
print(prediction)
print(y_test)

C:\Users\DELL\AppData\Local\Temp\ipykernel_19544\817588615.py:9: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


[5 5 5 ... 5 5 5]
[7 2 1 ... 4 5 6]


In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)

0.0892